In [1]:
import keras
from keras.layers import Dense
from keras.models import Sequential
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../src/')
# Project modules
import get_modeling_data
import model_evaluation.model_performance_functions as mpf
import features.win_rates as wr
import data_constants as dc
import warnings
warnings.filterwarnings('ignore')

C:\Users\Albert\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [24]:
train = get_modeling_data.get_train()
validation = get_modeling_data.get_validation()
train = train.fillna(0)
validation = validation.fillna(0)

In [25]:
champ_classes = pd.read_csv('../data/interim/champ_classes.csv', index_col=0)

In [26]:
for col in champ_classes.columns:
    train['100_' + col + '_cnt'] = 0
    train['200_' + col + '_cnt'] = 0
    validation['100_' + col + '_cnt'] = 0
    validation['200_' + col + '_cnt'] = 0

In [27]:
team_100_cols = ['100_TOP_SOLO', '100_MIDDLE_SOLO', '100_JUNGLE_NONE', '100_BOTTOM_DUO_CARRY', '100_BOTTOM_DUO_SUPPORT']
for lane_col in team_100_cols:
    train = pd.merge(train, champ_classes, how='left', left_on=lane_col, right_index=True)
    validation = pd.merge(validation, champ_classes, how='left', left_on=lane_col, right_index=True)
    for col in champ_classes.columns:
        train['100_' + col + '_cnt'] = train['100_' + col + '_cnt'] + train[col]
        train.drop([col], inplace=True, axis=1)
        validation['100_' + col + '_cnt'] = validation['100_' + col + '_cnt'] + validation[col]
        validation.drop([col], inplace=True, axis=1)

team_200_cols = ['200_TOP_SOLO', '200_MIDDLE_SOLO', '200_JUNGLE_NONE', '200_BOTTOM_DUO_CARRY', '200_BOTTOM_DUO_SUPPORT']
for lane_col in team_200_cols:
    train = pd.merge(train, champ_classes, how='left', left_on=lane_col, right_index=True)
    validation = pd.merge(validation, champ_classes, how='left', left_on=lane_col, right_index=True)
    for col in champ_classes.columns:
        train['200_' + col + '_cnt'] = train['200_' + col + '_cnt'] + train[col]
        train.drop([col], inplace=True, axis=1)
        validation['200_' + col + '_cnt'] = validation['200_' + col + '_cnt'] + validation[col]
        validation.drop([col], inplace=True, axis=1)


### Using only the count columns

In [28]:
cnt_cols = [x for x in train.columns if x[-3:] == 'cnt']
X_train = train[cnt_cols]
Y_train = train['team_100_win']
X_validation = validation[cnt_cols]
Y_validation = validation['team_100_win']

In [32]:
model = Sequential()
model.add(Dense(25, activation='sigmoid', input_shape=(X_train.shape[1],)))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, Y_train)
train_pred = model.predict(X_train)
print("Train ks and gini: " + str(mpf.ks_gini(Y_train, train_pred[:,0])))
print("Train accuracy: " + str(mpf.correct_prediction_rate(Y_train, train_pred[:,0])))
valid_pred = model.predict(X_validation)
print("Validation ks and gini: " + str(mpf.ks_gini(Y_validation, valid_pred[:,0])))
print("Validation accuracy: " + str(mpf.correct_prediction_rate(Y_validation, valid_pred[:,0])))

Epoch 1/1
171284/171284 [==============================] - 6s 37us/step - loss: 0.2503
Train ks and gini: {'ks': 0.05151882817460718, 'gini': 0.034438583887074346}
Train accuracy: 0.5219751990845614
Validation ks and gini: {'ks': 0.05414927529111058, 'gini': 0.035423347236456404}
Validation accuracy: 0.5200112097805335


In [31]:
X_train.shape

(171284, 36)

### Count columns along with champion indicators

In [34]:
lane_cols = ['100_TOP_SOLO', '100_JUNGLE_NONE', '100_MIDDLE_SOLO', '100_BOTTOM_DUO_CARRY', '100_BOTTOM_DUO_SUPPORT', '200_TOP_SOLO', '200_JUNGLE_NONE', '200_MIDDLE_SOLO', '200_BOTTOM_DUO_CARRY', '200_BOTTOM_DUO_SUPPORT']
X_train2 = train[cnt_cols + lane_cols]
X_validation2 = validation[cnt_cols + lane_cols]

champs = dc.get_champs_four_letters()

for champ in champs:
    X_train2[champ] = 0
    X_validation2[champ] = 0
    for col in lane_cols:
        if col[0:3] == '100':
            X_train2[champ][X_train2[col] == champ] = 1
            X_validation2[champ][X_validation2[col] == champ] = 1
        else:
            X_train2[champ][X_train2[col] == champ] = -1
            X_validation2[champ][X_validation2[col] == champ] = -1
X_train2.drop(lane_cols, inplace=True, axis=1)
X_validation2.drop(lane_cols, inplace=True, axis=1)

In [37]:
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_shape=(X_train2.shape[1],)))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train2, Y_train)
train_pred = model.predict(X_train2)
print("Train ks and gini: " + str(mpf.ks_gini(Y_train, train_pred[:,0])))
print("Train accuracy: " + str(mpf.correct_prediction_rate(Y_train, train_pred[:,0])))
valid_pred = model.predict(X_validation2)
print("Validation ks and gini: " + str(mpf.ks_gini(Y_validation, valid_pred[:,0])))
print("Validation accuracy: " + str(mpf.correct_prediction_rate(Y_validation, valid_pred[:,0])))

Epoch 1/1
171284/171284 [==============================] - 14s 82us/step - loss: 0.2524
Train ks and gini: {'ks': 0.1028008324063121, 'gini': 0.07210037330478783}
Train accuracy: 0.5486151654562014
Validation ks and gini: {'ks': 0.10563524742840258, 'gini': 0.0717467734403451}
Validation accuracy: 0.5485436042947471
